In [31]:
# Database connector
from db_connector import DatabaseConnector
# from urllib.parse import quote_plus # URL encode the password

# Data manipulation
import pandas as pd 
# import numpy as np
# import geopandas as gpd

In [32]:
# to retrieve login credential from defined config file
from config import DB_CONFIG

# Assuming DatabaseConnector is a class you've defined or imported
db_connector = DatabaseConnector(DB_CONFIG, use_sqlalchemy=True)
db_connector.connect()

2024-03-18 15:31:38,050 - INFO - Database connection successful (SQLAlchemy).


In [33]:
# Read the data from the database
query = "SELECT * FROM lookup_premise"

# Read the data from the database
df = pd.read_sql_query(query, db_connector.engine)

# Close the connection
db_connector.close_connection()

# Display the first few rows of the dataframe
df.head()

2024-03-18 15:31:40,240 - INFO - Database connection closed.


,premise_code,premise,address,premise_type,state,district
0,-1,,",",,,
1,2,PASAR BESAR IPOH,"JALAN LAKSAMANA,TAMAN JUBILEE,30300 IPOH, PERAK",Pasar Basah,Perak,Kinta
2,3,JUSCO AYER KEROH,"LOT 4991,MUKIM BUKIT BARU,75450 LEBUH AYER KER...",Pasar Raya / Supermarket,Melaka,Melaka Tengah
3,6,KEDAI RUNCIT TAFAZ MAJU PRESINT 9,"NO.5, JALAN P9B/1,PRESINT 9,62250 PUTRAJAYA",Kedai Runcit,W.P. Putrajaya,Wp Putrajaya
4,7,KEDAI RUNCIT SYAZ MAJU PRESINT 9,"NO.5, JALAN P9E/1,PRESINT 9,62250 PUTRAJAYA",Kedai Runcit,W.P. Putrajaya,Wp Putrajaya


In [34]:
# Read the data from the database
df.head(7)

,premise_code,premise,address,premise_type,state,district
0,-1,,",",,,
1,2,PASAR BESAR IPOH,"JALAN LAKSAMANA,TAMAN JUBILEE,30300 IPOH, PERAK",Pasar Basah,Perak,Kinta
2,3,JUSCO AYER KEROH,"LOT 4991,MUKIM BUKIT BARU,75450 LEBUH AYER KER...",Pasar Raya / Supermarket,Melaka,Melaka Tengah
3,6,KEDAI RUNCIT TAFAZ MAJU PRESINT 9,"NO.5, JALAN P9B/1,PRESINT 9,62250 PUTRAJAYA",Kedai Runcit,W.P. Putrajaya,Wp Putrajaya
4,7,KEDAI RUNCIT SYAZ MAJU PRESINT 9,"NO.5, JALAN P9E/1,PRESINT 9,62250 PUTRAJAYA",Kedai Runcit,W.P. Putrajaya,Wp Putrajaya
5,8,PASAR MINI D'MART PRESINT 8,"BLOK A T00/U01,JALAN P8/D,PRESINT 8, 62000 PUT...",Pasar Mini,W.P. Putrajaya,Wp Putrajaya
6,15,PASAR MINI KARIM,"NO, 7A,KAWASAN PASAR UTAN AJI,01000 KANGAR, PE...",Kedai Runcit,Perlis,Kangar


In [35]:
df["latitude"] = ""
df["longitude"] = ""
df.head(5)

,premise_code,premise,address,premise_type,state,district,latitude,longitude
0,-1,,",",,,,,
1,2,PASAR BESAR IPOH,"JALAN LAKSAMANA,TAMAN JUBILEE,30300 IPOH, PERAK",Pasar Basah,Perak,Kinta,,
2,3,JUSCO AYER KEROH,"LOT 4991,MUKIM BUKIT BARU,75450 LEBUH AYER KER...",Pasar Raya / Supermarket,Melaka,Melaka Tengah,,
3,6,KEDAI RUNCIT TAFAZ MAJU PRESINT 9,"NO.5, JALAN P9B/1,PRESINT 9,62250 PUTRAJAYA",Kedai Runcit,W.P. Putrajaya,Wp Putrajaya,,
4,7,KEDAI RUNCIT SYAZ MAJU PRESINT 9,"NO.5, JALAN P9E/1,PRESINT 9,62250 PUTRAJAYA",Kedai Runcit,W.P. Putrajaya,Wp Putrajaya,,


In [36]:
import pandas as pd
import requests
import config

# Assuming df is your DataFrame loaded with data
# Google Maps API key
Google_Maps_API_KEY = config.Google_Maps_API_KEY  # to remove/replace this later on!! 

def fetch_coordinates(api_key, address):
    """Fetch coordinates (latitude, longitude) for the given address using Google Maps Geocoding API."""
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    response = requests.get(base_url, params={'address': address, 'key': api_key})
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            latitude = data['results'][0]['geometry']['location']['lat']
            longitude = data['results'][0]['geometry']['location']['lng']
            return latitude, longitude
    return None, None

# Loop through the DataFrame and update with coordinates
for index, row in df.iterrows():
    if row['address'].strip():  # Skip empty addresses
        lat, lon = fetch_coordinates(api_key, row['address'])
        df.at[index, 'latitude'] = lat
        df.at[index, 'longitude'] = lon
        
    if (index + 1) % 500 == 0:  # Print update every 500 rows
        print(f"Updated {index + 1} rows with coordinates.")

# Final update status
print(f"Finished updating coordinates for all rows.")

# Optionally, print or inspect the first few rows to check updates
print(df.head())

Updated 500 rows with coordinates.
Updated 1000 rows with coordinates.
Updated 1500 rows with coordinates.
Updated 2000 rows with coordinates.
Updated 2500 rows with coordinates.
Finished updating coordinates for all rows.
   premise_code                            premise  \
0            -1                                      
1             2                   PASAR BESAR IPOH   
2             3                   JUSCO AYER KEROH   
3             6  KEDAI RUNCIT TAFAZ MAJU PRESINT 9   
4             7   KEDAI RUNCIT SYAZ MAJU PRESINT 9   

                                             address  \
0                                                  ,   
1    JALAN LAKSAMANA,TAMAN JUBILEE,30300 IPOH, PERAK   
2  LOT 4991,MUKIM BUKIT BARU,75450 LEBUH AYER KER...   
3        NO.5, JALAN P9B/1,PRESINT 9,62250 PUTRAJAYA   
4        NO.5, JALAN P9E/1,PRESINT 9,62250 PUTRAJAYA   

               premise_type           state       district  latitude  \
0                                          

In [66]:
# Display the first few rows of the DataFrame to check the new columns
print("First few rows of the processed DataFrame:")
print(df[['latitude', 'longitude']].head())

First few rows of the processed DataFrame:
   latitude   longitude
0      None        None
1  4.592759  101.083988
2  2.242987  102.290511
3  2.944309  101.675805
4  2.941244  101.675183


In [67]:
# Display the last few rows of the DataFrame to check the new columns
print("Last few rows of the processed DataFrame:")
print(df[['latitude', 'longitude']].tail())

Last few rows of the processed DataFrame:
      latitude   longitude
2705   3.14701  101.737343
2706  2.798548  103.485467
2707  2.821091  101.817199
2708  3.110393  101.636942
2709  3.075532  101.633965


In [53]:
# Check the number of unique place IDs
print(f"Unique latitudes: {df['latitude'].nunique()}")

# Check the number of unique formatted addresses
print(f"Unique longitude: {df['longitude'].nunique()}")

Unique latitudes: 2314
Unique longitude: 2311


In [54]:
# Check for missing data in the new columns
print(df[['latitude', 'longitude']].isnull().sum())


latitude     61
longitude    61
dtype: int64


In [61]:
# Assuming 'df' is your DataFrame
filtered_df = df[df['latitude'].isnull() & df['longitude'].isnull()]

# Now, 'filtered_df' contains only the rows where both 'latitude' and 'longitude' have null values.
filtered_df

,premise_code,premise,address,premise_type,state,district,latitude,longitude
0,-1,,",",,,,None,None
44,382,PASARAYA PKT PASIR TUMBOH,"G6582,LOT 11,KEDAI PASIR TUMBOH,",Pasar Raya / Supermarket,Kelantan,Kota Bharu,None,None
56,480,PASARAYA PANTAI TIMUR (BACHOK) SDN. BHD,"LOT 2076, MUKIM TEPUS,GUNUNG TIMUR,",Pasar Raya / Supermarket,Kelantan,Bachok,None,None
141,920,PASAR AWAM GEMAS,"NO. 1,PASAR BESAR GEMAS,",Pasar Basah,Negeri Sembilan,Gemas,None,None
237,1511,PASAR MINI BUDI,"NO. 24,KG. BUDI,",Pasar Mini,Kelantan,Bachok,None,None
...,...,...,...,...,...,...,...,...
2587,20772,MARI-MARI SUPERMARKET,"NO 7,8 & 9, JALAN SRI MERSING 11,PUSAT PERDAGA...",Pasar Raya / Supermarket,Johor,Mersing,None,None
2633,20819,KEDAI RUNCIT HJ. IBRAHIM,"NO 208,PTG. TOK MAHAT,",Kedai Runcit,Perak,Kerian,None,None
2634,20820,79 FRESH MART SDN BHD,"LOT 2526,TAMAN ROD,",Kedai Runcit,Perak,Hilir Perak,None,None
2684,20871,KITAI MART SDN BHD,"LOT 1273,SUBLOT 17-22,BLOCK 13,MENUAN LAND DIS...",Pasar Mini,Sarawak,Kapit,None,None


In [74]:
filtered_df.head()

,premise_code,premise,address,premise_type,state,district,latitude,longitude
0,-1,,",",,,,None,None


In [73]:
# save the updated data frame to a new file to a directory named "lookup_premise_geocoded" on desktop 
filtered_df.to_csv('C://Users//user//Documents//WQD7002 - FYP//Datasets//lookup_premise_geocode_challege.csv', index=True)

In [68]:
import pandas as pd
import requests

# Assuming df is your DataFrame loaded with data

Google_Maps_API_KEY = config.Google_Maps_API_KEY   # Replace 'abcdeY' with your actual Google Maps API key

def fetch_coordinates(api_key, premise, address, state, district):
    """
    Fetch coordinates (latitude, longitude) for the given address using Google Maps Geocoding API.
    Enhance search accuracy by including premise name, state, and district in the search query.
    """
    # Construct a more detailed address by including premise, state, and district if available
    detailed_address = f"{premise}, {address}, {district}, {state}"
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    response = requests.get(base_url, params={'address': detailed_address.strip(), 'key': Google_Maps_API_KEY})
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            latitude = data['results'][0]['geometry']['location']['lat']
            longitude = data['results'][0]['geometry']['location']['lng']
            return latitude, longitude
    return None, None

# Loop through the DataFrame and update with coordinates
for index, row in df.iterrows():
    # Check for rows with missing or empty latitude and longitude values
    if pd.isnull(row['latitude']) or row['latitude'] == '' or pd.isnull(row['longitude']) or row['longitude'] == '':
        # Use premise, address, state, and district for enhanced geocoding accuracy
        lat, lon = fetch_coordinates(Google_Maps_API_KEY, row['premise'], row['address'], row['state'], row['district'])
        if lat is not None and lon is not None:
            df.at[index, 'latitude'] = lat
            df.at[index, 'longitude'] = lon
            print(f"Updated coordinates for row {index}.")
        
    if (index + 1) % 500 == 0:  # Print update every 500 rows
        print(f"Updated {index + 1} rows with coordinates.")

# Final update status
print(f"Finished updating coordinates for all rows.")

# Optionally, print or inspect the first few rows to check updates
print(df.head())

Updated coordinates for row 44.
Updated coordinates for row 56.
Updated coordinates for row 141.
Updated coordinates for row 237.
Updated coordinates for row 261.
Updated coordinates for row 363.
Updated coordinates for row 408.
Updated coordinates for row 439.
Updated coordinates for row 445.
Updated 500 rows with coordinates.
Updated coordinates for row 549.
Updated coordinates for row 691.
Updated coordinates for row 823.
Updated coordinates for row 894.
Updated coordinates for row 904.
Updated coordinates for row 921.
Updated 1000 rows with coordinates.
Updated coordinates for row 1005.
Updated coordinates for row 1045.
Updated coordinates for row 1133.
Updated coordinates for row 1145.
Updated coordinates for row 1184.
Updated coordinates for row 1196.
Updated coordinates for row 1267.
Updated coordinates for row 1298.
Updated coordinates for row 1331.
Updated coordinates for row 1359.
Updated 1500 rows with coordinates.
Updated coordinates for row 1538.
Updated coordinates for ro

In [69]:
# Check for missing data in the new columns
print(df[['latitude', 'longitude']].isnull().sum())

latitude     1
longitude    1
dtype: int64


In [70]:
# Assuming 'df' is your DataFrame
filtered_df = df[df['latitude'].isnull() & df['longitude'].isnull()]

# Now, 'filtered_df' contains only the rows where both 'latitude' and 'longitude' have null values.
filtered_df

,premise_code,premise,address,premise_type,state,district,latitude,longitude
0,-1,,",",,,,None,None


In [71]:
# Check the number of unique place IDs
print(f"Unique latitudes: {df['latitude'].nunique()}")

# Check the number of unique formatted addresses
print(f"Unique longitude: {df['longitude'].nunique()}")

Unique latitudes: 2360
Unique longitude: 2357


In [72]:
# save the updated data frame to a new file to a directory named "lookup_premise_geocoded" on desktop 
df.to_csv('C://Users//user//Documents//WQD7002 - FYP//Datasets//lookup_premise_geocoded.csv', index=True)